In [1]:
import sys
import pandas as pd
import numpy as np
import json
import string
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/magupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
answers=pd.read_csv("data_stackOverflow/original_data/Answers.csv", encoding='latin-1')
questions=pd.read_csv("data_stackOverflow/original_data/Questions.csv", encoding='latin-1')
tags=pd.read_csv("data_stackOverflow/original_data/Tags.csv", encoding='latin-1')

In [3]:
questions_original=pd.read_csv("data_stackOverflow/original_data/Questions.csv", encoding='latin-1')

In [4]:
questions_original.loc[2]['Body']

"<p>Has anyone got experience creating <strong>SQL-based ASP.NET</strong> site-map providers?</p>\n\n<p>I've got the default XML file <code>web.sitemap</code> working properly with my Menu and <strong>SiteMapPath</strong> controls, but I'll need a way for the users of my site to create and modify pages dynamically.</p>\n\n<p>I need to tie page viewing permissions into the standard <code>ASP.NET</code> membership system as well.</p>\n"

In [5]:
with open('contractions.json') as json_file:
    contractions = json.load(json_file)
contractions

{"ain't": 'am not / are not',
 "aren't": 'are not / am not',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "'cause": 'because',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he had / he would',
 "he'd've": 'he would have',
 "he'll": 'he shall / he will',
 "he'll've": 'he shall have / he will have',
 "he's": 'he has / he is',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how has / how is',
 "i'd": 'I had / I would',
 "i'd've": 'I would have',
 "i'll": 'I shall / I will',
 "i'll've": 'I shall have / I will have',
 "i'm": 'I am',
 "i've": 'I have',
 "isn't": 'is not',
 "it'd": 'it had / it would',
 "it'd've": 'it would have',
 "it'll": 'it shall / it will',
 "it'll've": 'it shall have / it will have',
 "it's": 'it has / it is',
 "let's": 'l

In [6]:
questions.shape,tags.shape

((1264216, 7), (3750994, 2))

In [7]:
answers.head(2)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...


In [8]:
questions.head(2)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...


In [9]:
tags.head(2)

,Id,Tag
0,80,flex
1,80,actionscript-3


In [10]:
#drop unwanted columns
answers.drop(columns=['OwnerUserId','CreationDate','ParentId'],inplace=True)
questions.drop(columns=['OwnerUserId','CreationDate','ClosedDate'],inplace=True)

In [11]:
answers=answers.rename(columns={'Body': 'ans_body','Score': 'ans_score'})
questions=questions.rename(columns={'Body': 'ques_body','Score': 'ques_score','Title': 'ques_title'})

In [12]:
answers.columns,questions.columns,tags.columns

(Index(['Id', 'ans_score', 'ans_body'], dtype='object'),
 Index(['Id', 'ques_score', 'ques_title', 'ques_body'], dtype='object'),
 Index(['Id', 'Tag'], dtype='object'))

In [13]:
#questions and tags dataframe have same set of Ids.
difference_1=set(answers['Id']).difference(set(questions['Id']))
difference_2=set(answers['Id']).difference(set(tags['Id']))
difference_3=set(questions['Id']).difference(set(tags['Id']))
len(difference_1),len(difference_2),len(difference_3)

(2014516, 2014516, 0)

In [14]:
#removes paragraph and next line symbols
def remove_symbols_numbers(text):
    text=text.replace('\n', ' ')
    text=text.replace('<p>', ' ')
    text=text.replace('</p>', ' ')
    text=text.translate(str.maketrans('', '','0123456789'))
    #text=re.sub(r'[^\w]', ' ', text)
    #print("symbols done")
    return text

#removes html tags
def remove_html(text,flag):
    if (flag=='body'):
        text=BeautifulSoup(text,'html.parser')
    return text

#Remove contracted forms of words in statements
def decontract(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    #print("decontraction done")
    return text

#Remove punctuations from statements
def remove_punctuations(text):
    text=text.translate(str.maketrans(string.punctuation,' '*32))
    #print("punctuations done")
    return text

#Remove frequently occurring helping words i.e stopwords
stop = stopwords.words('english')
def remove_stopwords(text):
    text=' '.join([word for word in text.split() if word not in (stop)])
    #print("stopwords done")
    return text

#lemmatizatiiiion
def lemmatize(text):
    text=' '.join([stemmer.stem(y) for y in text.split()])
    #print("lemmatization done")
    return text

#Tex preprocessing function
def text_preprocessing(text,flag):
    symbol_free_text=remove_symbols_numbers(text)
    html_free_text=remove_html(text,flag)
    contraction_free_text=decontract(symbol_free_text)
    punctuation_free_text=remove_punctuations(contraction_free_text)
    sw_free_text=remove_stopwords(punctuation_free_text)
    preprocessed_text=lemmatize(sw_free_text)
    return preprocessed_text

In [15]:
#answers['ans_body']=answers['ans_body'].apply(lambda x : text_preprocessing(x.lower()))
questions['ques_body']=questions['ques_body'].apply(lambda x : text_preprocessing(x.lower(),"body"))
questions['ques_title']=questions['ques_title'].apply(lambda x : text_preprocessing(x.lower(),"title"))

In [16]:
questions.to_csv("data_stackOverflow/text_preprocessed_data.csv",index=False)

In [17]:
questions=pd.read_csv("data_stackOverflow/text_preprocessed_data.csv")

In [18]:
answers.head(1)

,Id,ans_score,ans_body
0,92,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."


In [19]:
questions.head(1)

,Id,ques_score,ques_title,ques_body
0,80,26,sqlstatement execut multipl queri one statement,i written databas generat script href http en ...


In [21]:
questions["ques_score"].describe().apply(lambda x: int(x))

count    1264216
mean           1
std           13
min          -73
25%            0
50%            0
75%            1
max         5190
Name: ques_score, dtype: int64

In [22]:
questions['ques_score_bin'] = pd.qcut(questions['ques_score'],labels=["bad", "medium", "good"], q=3,duplicates= 'drop')
questions['ques_score_bin'].value_counts()

bad       670159
good      313015
medium    281042
Name: ques_score_bin, dtype: int64

In [23]:
questions=pd.get_dummies(questions, columns=['ques_score_bin'], prefix='ques_score')

In [24]:
questions.head(1)

,Id,ques_score,ques_title,ques_body,ques_score_bad,ques_score_medium,ques_score_good
0,80,26,sqlstatement execut multipl queri one statement,i written databas generat script href http en ...,0,0,1


In [25]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [26]:
tags['Tag'].unique(),len(tags['Tag'].unique())

(array(['flex', 'actionscript-3', 'air', ..., 'union-types', 'voce',
        'docker-windows'], dtype=object),
 37035)

In [27]:
tags['Tag'].value_counts()

javascript                 124155
java                       115212
c#                         101186
php                         98808
android                     90659
                            ...  
escalante                       1
anonymousidentification         1
uppaal                          1
sql-server-2012-web             1
creation-pattern                1
Name: Tag, Length: 37034, dtype: int64

In [28]:
tags_final = pd.DataFrame() 
tags_final=tags.apply(lambda x: x.mask(x.map(x.value_counts())<np.percentile(tags['Tag'].value_counts().values,99.95), 'other'))
tags_final['Id']=tags['Id']

In [29]:
tags_final["Tag"].value_counts()

other            2651071
javascript        124155
java              115212
c#                101186
php                98808
android            90659
jquery             78542
python             64601
html               58976
c++                47591
ios                47009
mysql              42464
css                42308
sql                35782
asp.net            29970
objective-c        26922
ruby-on-rails      25789
.net               24059
c                  23238
iphone             21539
Name: Tag, dtype: int64

In [30]:
tags_final_one_hot=pd.get_dummies(tags_final, columns=['Tag'], prefix='tag')

In [31]:
tags_final_one_hot.head(2)

,Id,tag_.net,tag_android,tag_asp.net,tag_c,tag_c#,tag_c++,tag_css,tag_html,tag_ios,...,tag_java,tag_javascript,tag_jquery,tag_mysql,tag_objective-c,tag_other,tag_php,tag_python,tag_ruby-on-rails,tag_sql
0,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [32]:
tags_final['Tag']=tags_final['Tag'].astype(str)
grouped_tags_1 = tags_final_one_hot.groupby("Id").first()
grouped_tags_1['maintag']=tags_final.groupby("Id")["Tag"].first()
grouped_tags_1['all_tags']=tags_final.groupby("Id")['Tag'].apply(lambda tags: '/'.join(tags))
grouped_tags_1.head(2)

,tag_.net,tag_android,tag_asp.net,tag_c,tag_c#,tag_c++,tag_css,tag_html,tag_ios,tag_iphone,...,tag_jquery,tag_mysql,tag_objective-c,tag_other,tag_php,tag_python,tag_ruby-on-rails,tag_sql,maintag,all_tags
Id,,,,,,,,,,,,,,,,,,,,,
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,other,other/other/other
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,other,other/other/other/other


In [33]:
grouped_tags_2 = tags_final_one_hot.groupby("Id").sum()
grouped_tags_2.head(2)

,tag_.net,tag_android,tag_asp.net,tag_c,tag_c#,tag_c++,tag_css,tag_html,tag_ios,tag_iphone,tag_java,tag_javascript,tag_jquery,tag_mysql,tag_objective-c,tag_other,tag_php,tag_python,tag_ruby-on-rails,tag_sql
Id,,,,,,,,,,,,,,,,,,,,
80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0


In [34]:
grouped_tags_2[grouped_tags_2 > 1] = 1

In [35]:
grouped_tags_2.head(2)

,tag_.net,tag_android,tag_asp.net,tag_c,tag_c#,tag_c++,tag_css,tag_html,tag_ios,tag_iphone,tag_java,tag_javascript,tag_jquery,tag_mysql,tag_objective-c,tag_other,tag_php,tag_python,tag_ruby-on-rails,tag_sql
Id,,,,,,,,,,,,,,,,,,,,
80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [36]:
grouped_tags_1.shape,grouped_tags_2.shape

((1264216, 22), (1264216, 20))

In [37]:
df_1 = questions.merge(grouped_tags_1, on='Id')
df_2 = questions.merge(grouped_tags_2, on='Id')

In [38]:
df_1.drop(["Id","ques_score"],axis=1,inplace=True)
df_2.drop(["Id","ques_score"],axis=1,inplace=True)

In [39]:
print("df_1 shape = ",df_1.shape)
print("df_2 shape = ",df_2.shape)

df_1 shape =  (1264216, 27)
df_2 shape =  (1264216, 25)


In [40]:
df_1.head(2)

,ques_title,ques_body,ques_score_bad,ques_score_medium,ques_score_good,tag_.net,tag_android,tag_asp.net,tag_c,tag_c#,...,tag_jquery,tag_mysql,tag_objective-c,tag_other,tag_php,tag_python,tag_ruby-on-rails,tag_sql,maintag,all_tags
0,sqlstatement execut multipl queri one statement,i written databas generat script href http en ...,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,other,other/other/other
1,good branch merg tutori tortoisesvn,realli good tutori explain href http svnbook r...,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,other,other/other/other/other


In [41]:
df_2.head(2)

,ques_title,ques_body,ques_score_bad,ques_score_medium,ques_score_good,tag_.net,tag_android,tag_asp.net,tag_c,tag_c#,...,tag_java,tag_javascript,tag_jquery,tag_mysql,tag_objective-c,tag_other,tag_php,tag_python,tag_ruby-on-rails,tag_sql
0,sqlstatement execut multipl queri one statement,i written databas generat script href http en ...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,good branch merg tutori tortoisesvn,realli good tutori explain href http svnbook r...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [42]:
df_1['maintag'].value_counts()

other            424442
javascript       118826
java             114208
c#               100192
php               92015
android           71963
python            62622
c++               45318
ios               36618
jquery            31429
ruby-on-rails     22556
html              22276
sql               22094
mysql             20254
c                 18987
iphone            17165
asp.net           13329
css               11288
objective-c       10055
.net               8546
nan                  33
Name: maintag, dtype: int64

# Removing numbers has led to this kind of text which ignores the version of the framework

In [43]:
questions_original.iloc[100]['Title'],df_2.iloc[100]['ques_title']

('What are the useful new ASP.NET features in the .NET Framework 3.5?',
 'use new asp net featur net framework')

# All negations have been removed and thus meaning of sentence has changed

In [44]:
questions_original.iloc[100]['Body'],df_1.iloc[100]['ques_body']

("<p>I've kept up to date with new features in the C# language as it's moved from version 1 through version 3.  I haven't done such a good job keeping up to date with ASP.NET.  I feel like some of the post version 1 features are not so good (e.g. the AJAX framework) or are just not that useful to me (e.g. the membership framework).  Can anyone recommend any new killer ASP.NET features that might have gone unnoticed?</p>\n",
 'i kept date new featur c languag move version version done good job keep date asp net feel like post version featur good e g ajax framework use e g membership framework anyon recommend new killer asp net featur might gone unnot')

In [45]:
#2 duplicates present
print('Duplicate entries: {}'.format(df_1.duplicated().sum()))
print('Dupplicate entries: {}'.format(df_2.duplicated().sum()))

Duplicate entries: 4
Dupplicate entries: 5


In [46]:
#duplicates dropped
df_1.drop_duplicates( keep='first', inplace=True)
df_2.drop_duplicates( keep='first', inplace=True)

In [52]:
print("df_1 shape = ",df_1.shape)
print("df_2 shape = ",df_2.shape)

df_1 shape =  (1264212, 27)
df_2 shape =  (1264211, 25)


In [48]:
print('Duplicate entries: {}'.format(df_1.duplicated().sum()))
print('Dupplicate entries: {}'.format(df_2.duplicated().sum()))

Duplicate entries: 0
Dupplicate entries: 0


In [49]:
df_1.to_csv("data_stackOverflow/final_dataframe_with_1tag_onehot.csv")
df_2.to_csv("data_stackOverflow/final_dataframe_with_multipletags_onehot.csv")
#answers.to_csv("data_stackOverflow/preprocessed_answers.csv")

In [50]:
# No missing value present 
print(df_1.isnull().sum())
print(df_2.isnull().sum())

ques_title           15
ques_body             1
ques_score_bad        0
ques_score_medium     0
ques_score_good       0
tag_.net              0
tag_android           0
tag_asp.net           0
tag_c                 0
tag_c#                0
tag_c++               0
tag_css               0
tag_html              0
tag_ios               0
tag_iphone            0
tag_java              0
tag_javascript        0
tag_jquery            0
tag_mysql             0
tag_objective-c       0
tag_other             0
tag_php               0
tag_python            0
tag_ruby-on-rails     0
tag_sql               0
maintag               0
all_tags              0
dtype: int64
ques_title           15
ques_body             1
ques_score_bad        0
ques_score_medium     0
ques_score_good       0
tag_.net              0
tag_android           0
tag_asp.net           0
tag_c                 0
tag_c#                0
tag_c++               0
tag_css               0
tag_html              0
tag_ios               0
tag

In [51]:
print(df_1['maintag'].isnull().sum())

0
